In [29]:
import json
import re
import requests
from bs4 import BeautifulSoup

In [6]:
transcript_url = 'https://www.officialhacksandwonks.com/sse-37th-ld-debate-2022#transcript'
transcript = BeautifulSoup(requests.get(transcript_url).text)

In [13]:
transcript.find_all('p')[0].text

"[00:00:09] Crystal Fincher: Welcome everyone to the South Seattle Emerald's 2022 General Election Candidate Debate. My name is Crystal Fincher. I'm a political consultant and the host of the Hacks & Wonks radio show and podcast, and I'm honored to welcome you all to tonight's debate. I'm also excited to hear from our guests running for State Representative Position 2 in the 37th Legislative District."

In [15]:
pattern = re.compile(r'\[\d+:\d+:\d+]\s(?P<name>.*):(?P<response>.*)')

In [27]:
responses = []
name = ''
for paragraph in transcript.find_all('p'):
    match = pattern.match(paragraph.text)
    if match:
        name = match.group('name')
        responses.append((name, match.group('response')))
    else:
        responses.append((name, paragraph.text))

In [36]:
cards = []
speeches = set()
for row in responses:
    speeches.add(row[0])
    cards.append(row[1])
with open('/Users/abrahamepton/code/flow/src/data/37th_debate.json', 'w+') as fh:
    fh.write(
        json.dumps(
            [{'text': response[1].strip(), 'speech': response[0].strip()} for response in responses]
        )
    )